In [1]:
# Inicializacion

In [2]:
# Importar bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import datetime, timedelta, date

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

#from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR

from scipy.stats import norm

from sklearn.metrics import make_scorer
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
import sklearn.metrics as metrics

import lightgbm as lgb

In [3]:
# calculo del rmse
def rmse(actual, predict):
    predict = np.array(predict)
    actual = np.array(actual)
    distance = predict - actual
    square_distance = distance**2
    mean_square_distance = square_distance.mean()
    score = np.sqrt(mean_square_distance)
    return score

rmse_score = make_scorer(rmse, greater_is_better = False)

In [4]:
# Regression metrics
def regression_results(y_true, y_pred):    
    explained_variance = metrics.explained_variance_score(y_true, y_pred)
    mean_absolute_error = metrics.mean_absolute_error(y_true, y_pred) 
    mse = metrics.mean_squared_error(y_true, y_pred) 
    mean_squared_log_error = metrics.mean_squared_log_error(y_true, y_pred)
    median_absolute_error = metrics.median_absolute_error(y_true, y_pred)
    r2 = metrics.r2_score(y_true, y_pred)    
    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(rmse(y_true, y_pred),4))

In [5]:
# Curva ROC
def roc_curve_func(model, Xtest, ytest):
    ypred_proba = model.predict_proba(Xtest)

    fpr, tpr, thr = roc_curve(ytest, ypred_proba[:,1])

    df = pd.DataFrame(dict(fpr=fpr, tpr=tpr, thr=thr))

    plt.axis([0, 1.01, 0, 1.01])
    plt.xlabel('1 - Specificty')
    plt.ylabel('TPR / Sensitivity')
    plt.title('ROC Curve')
    plt.plot(df['fpr'],df['tpr'])
    plt.plot(np.arange(0,1, step =0.01), np.arange(0,1, step =0.01))
    plt.show()

In [6]:
# Importar datos

In [7]:
route_train = 'train_data.parquet'
train = pd.read_parquet(route_train)
train.head()

,sku,date,sold_quantity,current_price,currency,listing_type,shipping_logistic_type,shipping_payment,minutes_active
0,464801,2021-02-01,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
1,464801,2021-02-02,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
2,464801,2021-02-03,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
3,464801,2021-02-04,0,156.78,REA,classic,fulfillment,free_shipping,1440.0
4,464801,2021-02-05,1,156.78,REA,classic,fulfillment,free_shipping,1440.0


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37660279 entries, 0 to 37660278
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   sku                     int64  
 1   date                    object 
 2   sold_quantity           int64  
 3   current_price           float64
 4   currency                object 
 5   listing_type            object 
 6   shipping_logistic_type  object 
 7   shipping_payment        object 
 8   minutes_active          float64
dtypes: float64(2), int64(2), object(5)
memory usage: 2.5+ GB


In [9]:
len(train['sku'].unique())

660916

In [10]:
train_select = train.loc[train['sku'] <= 100000].copy()

In [11]:
train_select.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5700995 entries, 118 to 37660278
Data columns (total 9 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   sku                     int64  
 1   date                    object 
 2   sold_quantity           int64  
 3   current_price           float64
 4   currency                object 
 5   listing_type            object 
 6   shipping_logistic_type  object 
 7   shipping_payment        object 
 8   minutes_active          float64
dtypes: float64(2), int64(2), object(5)
memory usage: 435.0+ MB


In [12]:
del train

In [13]:
route_metadata = 'items_static_metadata_full.jl'
metadata = pd.read_json(route_metadata, lines = True)

In [14]:
data = pd.merge(train_select, metadata, how = "left", on = ["sku"])

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5700995 entries, 0 to 5700994
Data columns (total 15 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   sku                     int64  
 1   date                    object 
 2   sold_quantity           int64  
 3   current_price           float64
 4   currency                object 
 5   listing_type            object 
 6   shipping_logistic_type  object 
 7   shipping_payment        object 
 8   minutes_active          float64
 9   item_domain_id          object 
 10  item_id                 int64  
 11  item_title              object 
 12  site_id                 object 
 13  product_id              object 
 14  product_family_id       object 
dtypes: float64(2), int64(3), object(10)
memory usage: 695.9+ MB


In [16]:
del metadata

In [17]:
col_categoricas = [x for x in data.columns if data[x].dtype == 'object']
for col in col_categoricas:
    data[col] = data[col].astype(str)

data = data.drop(columns = ['product_id', 'product_family_id'])

data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5700995 entries, 0 to 5700994
Data columns (total 13 columns):
 #   Column                  Dtype  
---  ------                  -----  
 0   sku                     int64  
 1   date                    object 
 2   sold_quantity           int64  
 3   current_price           float64
 4   currency                object 
 5   listing_type            object 
 6   shipping_logistic_type  object 
 7   shipping_payment        object 
 8   minutes_active          float64
 9   item_domain_id          object 
 10  item_id                 int64  
 11  item_title              object 
 12  site_id                 object 
dtypes: float64(2), int64(3), object(8)
memory usage: 608.9+ MB


In [18]:
data['site_id'].value_counts()

MLB    2925534
MLM    2284265
MLA     491196
Name: site_id, dtype: int64

In [19]:
data['item_domain_id'].value_counts()

MLB-SUPPLEMENTS             118861
MLM-CELLPHONE_COVERS         91990
MLM-T_SHIRTS                 38829
MLB-HAIR_TREATMENTS          38439
MLM-SNEAKERS                 36830
                             ...  
MLM-BICYCLE_V_BRAKE_SETS         8
MLB-BABY_BATHTUB_SEATS           8
MLA-CHISELS                      6
MLB-SKATEBOARD_TRUCKS            3
MLB-OVENS                        3
Name: item_domain_id, Length: 6213, dtype: int64

In [20]:
# Cantidad de tipos de producto
len(data['item_domain_id'].unique())

6213

In [21]:
data['listing_type'].value_counts()

classic    2867777
premium    2833218
Name: listing_type, dtype: int64

In [22]:
# Cantidad de tipos de envio
len(data['listing_type'].unique())

2

In [23]:
data['shipping_payment'].value_counts()

paid_shipping    3781786
free_shipping    1919209
Name: shipping_payment, dtype: int64

In [24]:
# Cantidad de tipos de envio
len(data['shipping_payment'].unique())

2

In [25]:
data['shipping_logistic_type'].value_counts()

fulfillment      4906444
drop_off          418667
cross_docking     375884
Name: shipping_logistic_type, dtype: int64

In [26]:
# Cantidad de tipos de logistica
len(data['shipping_logistic_type'].unique())

3

In [27]:
# Comprimir todo en un dataset donde cada fila es un SKU y las columnas son series de tiempo de las variables de interes

In [28]:
list_cols = ['sku']
list_fechas = data['date'].unique().tolist()
list_cols.extend(list_fechas)
list_sku = data['sku'].unique().tolist()

In [29]:
data_ventas = pd.pivot_table(data, values = 'sold_quantity', index = 'sku', columns = 'date')
data_ventas = data_ventas.fillna(value = 0)
data_ventas

date,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,...,2021-03-22,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29,2021-03-30,2021-03-31
sku,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,30.0,18.0,31.0,19.0,20.0,13.0,9.0,22.0,24.0,31.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Tirar data para liberar memoria
# del data

In [31]:
# Armar funcion que rellene la matriz y_pred con los valores de probabilidad normal inversa

# Z = (x - u)/std
# x = # del dia (0 a 30), indice de la columna
# u = dia medio donde se termina el stock, data_ventas['mean_days_to_break_stock']
# std = desvio estandar de donde se termina el stock, data_ventas['std_days_to_break_stock']
# Rellenar cada celda con norm_dist.cdf(Z)


In [32]:
# funcion que calcula la probabilidad de quedarse sin inventario para un dia determinado

def calcular_probas(data, rango_analisis, dia_final, rango_estimacion):
    
    # tomar una ventana de rango_dias para atras desde dia_final para calcular la media y desvio de ventas por sku
    mean_sales = data.loc[:, data.columns[dia_final - rango_analisis: dia_final]].mean(axis = 1)
    mean_days_to_break_stock = data.loc[:, 'target_stock'] / mean_sales
    std_days_to_break_stock = data.loc[:, data.columns[dia_final - rango_analisis: dia_final]].std(axis = 1)
    std_days_to_break_stock = std_days_to_break_stock / mean_sales
    
    # acomodar valores con errores
    mean_days_to_break_stock.replace(to_replace = np.inf, value = 1000, inplace = True)
    std_days_to_break_stock.replace(to_replace = np.inf, value = 10, inplace = True)
    mean_days_to_break_stock.replace(to_replace = np.nan, value = 0, inplace = True)
    std_days_to_break_stock.replace(to_replace = np.nan, value = 1, inplace = True)
    
    # crear variables de salida
    y_pred = pd.DataFrame(0, index = data.index, columns = range(0, rango_estimacion))
    y_pred_accum = pd.DataFrame(0, index = data.index, columns = range(0, rango_estimacion))
    
    # correr ciclo y calcular probabilidades con dist normal para cada día
    for i in y_pred.columns.tolist():
        if i == 0:
            probs = norm.cdf((np.full((data.shape[0]), i) - mean_sales) / std_days_to_break_stock)
            y_pred.loc[:, i] = np.around(probs, decimals = 3)
            y_pred_accum.loc[:, i] = np.around(probs, decimals = 3)
        else:
            probs = norm.cdf((np.full((data.shape[0]), i) - mean_sales) / std_days_to_break_stock)
            probs_ant = norm.cdf((np.full((data.shape[0]), i - 1) - mean_sales) / std_days_to_break_stock)
            delta_probs = probs - probs_ant
            delta_probs = np.around(delta_probs, decimals = 3)
            y_pred.loc[:, i] = delta_probs
            y_pred_accum.loc[:, i] = np.around(probs, decimals = 3)
    
    return y_pred, y_pred_accum


In [33]:
# Crear dataset con la variable respuesta para correr test y validar modelo

In [34]:
test_data_ventas = data_ventas.loc[:, data_ventas.columns < '2021-03-01'].copy()
test_data_ventas

date,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,...,2021-02-19,2021-02-20,2021-02-21,2021-02-22,2021-02-23,2021-02-24,2021-02-25,2021-02-26,2021-02-27,2021-02-28
sku,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,...,0.0,0.0,1.0,1.0,12.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,30.0,18.0,31.0,19.0,20.0,13.0,9.0,22.0,24.0,31.0,...,23.0,16.0,11.0,28.0,20.0,14.0,22.0,25.0,15.0,16.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
test_data_ventas['target_stock'] = data_ventas.loc[:, data_ventas.columns < '2021-03-01'].sum(axis = 1)
test_data_ventas

date,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,...,2021-02-20,2021-02-21,2021-02-22,2021-02-23,2021-02-24,2021-02-25,2021-02-26,2021-02-27,2021-02-28,target_stock
sku,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,5.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,...,0.0,1.0,1.0,12.0,0.0,1.0,0.0,1.0,0.0,35.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,30.0,18.0,31.0,19.0,20.0,13.0,9.0,22.0,24.0,31.0,...,16.0,11.0,28.0,20.0,14.0,22.0,25.0,15.0,16.0,556.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
rango_analisis = 15
dia_final = 28
rango_estimacion = 31
test_y_pred, test_y_pred_accum = calcular_probas(test_data_ventas, rango_analisis, dia_final, rango_estimacion)

In [38]:
test_y_pred

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
sku,,,,,,,,,,,,,,,,,,,,,
0,0.500,0.341,0.136,0.021,0.001,0.000,0.000,0.000,0.000,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.439,0.182,0.158,0.112,0.064,0.030,0.011,0.003,0.001,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.500,0.341,0.136,0.021,0.001,0.000,0.000,0.000,0.000,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.462,0.189,0.157,0.104,0.055,0.023,0.008,0.002,0.000,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.256,0.169,0.185,0.162,0.115,0.066,0.030,0.011,0.003,0.001,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,0.460,0.150,0.135,0.105,0.072,0.042,0.022,0.010,0.004,0.001,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,0.500,0.341,0.136,0.021,0.001,0.000,0.000,0.000,0.000,0.000,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
test_y_pred_accum

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
sku,,,,,,,,,,,,,,,,,,,,,
0,0.500,0.841,0.977,0.999,1.000,1.000,1.000,1.000,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.439,0.621,0.779,0.890,0.954,0.984,0.995,0.999,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.500,0.841,0.977,0.999,1.000,1.000,1.000,1.000,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.462,0.650,0.808,0.912,0.967,0.990,0.997,0.999,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.256,0.426,0.610,0.773,0.888,0.954,0.984,0.996,0.999,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99997,0.460,0.609,0.744,0.850,0.921,0.964,0.985,0.995,0.998,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99998,0.500,0.841,0.977,0.999,1.000,1.000,1.000,1.000,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [40]:
# Crear funcion que calcule cuando efectivamente se consumio el inventario del mes 2; dar probabilidades 0/1 para poder sacar el RMSE

In [41]:
# consumo diario del mes 2
test_consumo_real = data_ventas.loc[:, (data_ventas.columns >= '2021-03-01') & (data_ventas.columns <= '2021-03-31')].copy()
test_consumo_real.columns = range(0, len(test_consumo_real.columns))
test_consumo_real

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
sku,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,1.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0
1,2.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,39.0,22.0,22.0,4.0,25.0,27.0,22.0,27.0,38.0,34.0,...,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
99998,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
# Consumo real acumulado del mes 2
test_consumo_real_acc = pd.DataFrame(0, index = test_consumo_real.index, columns = range(0, len(test_consumo_real.columns)))

for i in test_consumo_real_acc.columns.tolist():
    if i == 0 :
        test_consumo_real_acc.loc[:, i] = test_consumo_real.loc[:, i].copy()
    else:
        test_consumo_real_acc.loc[:, i] = test_consumo_real.loc[:, i].copy() + test_consumo_real_acc.loc[:, i - 1]

test_consumo_real_acc

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
sku,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,4.0,8.0,10.0,10.0,11.0,11.0,11.0,11.0,11.0
1,2.0,2.0,2.0,2.0,3.0,3.0,3.0,4.0,4.0,4.0,...,14.0,14.0,14.0,14.0,14.0,14.0,16.0,16.0,16.0,17.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,2.0,3.0,3.0,3.0,3.0,4.0,4.0,4.0,4.0,4.0,...,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,39.0,61.0,83.0,87.0,112.0,139.0,161.0,188.0,226.0,260.0,...,582.0,582.0,582.0,582.0,582.0,582.0,582.0,582.0,582.0,582.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,2.0,3.0,4.0,6.0,...,8.0,8.0,8.0,8.0,8.0,8.0,10.0,10.0,10.0,10.0
99998,0.0,0.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,...,7.0,7.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0,8.0


In [46]:
# Matriz binaria 0 (hay inventario), 1 (se agoto el inventario)

test_y_real = pd.DataFrame(0, index = test_consumo_real.index, columns = range(0, len(test_consumo_real.columns)))

for i in test_y_real.columns.tolist():
    test = test_data_ventas['target_stock'] < test_consumo_real_acc.loc[:, i]
    test_y_real.loc[:, i] = test.astype('float64')

test_y_real

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
sku,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99998,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [47]:
test_y_pred_accum

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
sku,,,,,,,,,,,,,,,,,,,,,
0,0.500,0.841,0.977,0.999,1.000,1.000,1.000,1.000,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.439,0.621,0.779,0.890,0.954,0.984,0.995,0.999,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.500,0.841,0.977,0.999,1.000,1.000,1.000,1.000,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.462,0.650,0.808,0.912,0.967,0.990,0.997,0.999,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.256,0.426,0.610,0.773,0.888,0.954,0.984,0.996,0.999,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99997,0.460,0.609,0.744,0.850,0.921,0.964,0.985,0.995,0.998,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99998,0.500,0.841,0.977,0.999,1.000,1.000,1.000,1.000,1.000,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [48]:
test_y_real.shape

(100001, 31)

In [49]:
test_y_pred_accum.shape

(100001, 31)

In [50]:
# Evaluacion del modelo

In [51]:
regression_results(test_y_real, test_y_pred_accum)

explained_variance:  -0.2029
mean_squared_log_error:  0.306
r2:  -4.2488
MAE:  0.6505
MSE:  0.6281
RMSE:  0.7925


In [52]:
rmse(test_y_real, test_y_pred_accum)

0.7925408823093965

In [53]:
# Crear dataset con la variable respuesta para tener la predicción final

In [54]:
route_target = 'test_data.csv'
target_stock = pd.read_csv(route_target, delimiter = ',')

In [55]:
target_stock.head()

,sku,target_stock
0,464801,3
1,645793,4
2,99516,8
3,538100,8
4,557191,10


In [56]:
target_stock.index = target_stock['sku']
target_stock = target_stock.drop(columns = 'sku')
target_stock.sort_index(inplace = True)
target_stock.head()

,target_stock
sku,
0,1
1,13
2,10
3,3
5,12


In [57]:
data_ventas = pd.merge(data_ventas, target_stock, on = 'sku', how = 'left')
data_ventas

,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,...,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29,2021-03-30,2021-03-31,target_stock
sku,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,13.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,30.0,18.0,31.0,19.0,20.0,13.0,9.0,22.0,24.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [58]:
data_ventas['target_stock'].fillna(value = 0, inplace = True)
data_ventas

,2021-02-01,2021-02-02,2021-02-03,2021-02-04,2021-02-05,2021-02-06,2021-02-07,2021-02-08,2021-02-09,2021-02-10,...,2021-03-23,2021-03-24,2021-03-25,2021-03-26,2021-03-27,2021-03-28,2021-03-29,2021-03-30,2021-03-31,target_stock
sku,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,4.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,13.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,10.0
3,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,30.0,18.0,31.0,19.0,20.0,13.0,9.0,22.0,24.0,31.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,127.0
99997,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0
99998,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0


In [59]:
data_ventas.columns[29:59]

Index(['2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-06',
       '2021-03-07', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11',
       '2021-03-12', '2021-03-13', '2021-03-14', '2021-03-15', '2021-03-16',
       '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-20', '2021-03-21',
       '2021-03-22', '2021-03-23', '2021-03-24', '2021-03-25', '2021-03-26',
       '2021-03-27', '2021-03-28', '2021-03-29', '2021-03-30', '2021-03-31'],
      dtype='object')

In [60]:
# correr funcion para calcular probabilidades a futuro

rango_analisis = 30
dia_final = 59
rango_estimacion = 30
y_pred, y_pred_accum = calcular_probas(data_ventas, rango_analisis, dia_final, rango_estimacion)

In [61]:
y_pred

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
sku,,,,,,,,,,,,,,,,,,,,,
0,0.440,0.163,0.147,0.112,0.072,0.039,0.018,0.007,0.002,0.001,...,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.391,0.219,0.188,0.120,0.057,0.020,0.005,0.001,0.000,0.000,...,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.498,0.072,0.070,0.066,0.060,0.052,0.044,0.036,0.029,0.022,...,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.498,0.072,0.070,0.066,0.060,0.052,0.044,0.036,0.029,0.022,...,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.493,0.104,0.097,0.085,0.070,0.053,0.038,0.025,0.016,0.009,...,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.126,0.01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,0.433,0.198,0.168,0.111,0.057,0.023,0.007,0.002,0.000,0.000,...,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,0.451,0.180,0.155,0.108,0.062,0.029,0.011,0.003,0.001,0.000,...,0.000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
y_pred_accum

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
sku,,,,,,,,,,,,,,,,,,,,,
0,0.440,0.603,0.750,0.861,0.933,0.972,0.990,0.997,0.999,1.000,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.391,0.609,0.798,0.918,0.974,0.994,0.999,1.000,1.000,1.000,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.498,0.570,0.640,0.706,0.766,0.818,0.862,0.898,0.927,0.949,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.498,0.570,0.640,0.706,0.766,0.818,0.862,0.898,0.927,0.949,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.493,0.597,0.694,0.780,0.849,0.903,0.941,0.966,0.981,0.991,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.99,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99997,0.433,0.632,0.800,0.911,0.968,0.991,0.998,1.000,1.000,1.000,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99998,0.451,0.631,0.786,0.894,0.956,0.985,0.996,0.999,1.000,1.000,...,1.00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [63]:
y_pred.sum(axis = 1).mean()

0.9973817261827383

In [64]:
y_pred.sum(axis = 1)

sku
0         1.001
1         1.001
2         0.998
3         0.998
4         0.999
          ...  
99996     1.000
99997     0.999
99998     1.000
99999     1.000
100000    1.000
Length: 100001, dtype: float64

In [65]:
y_pred_accum.loc[:, 29].mean()

0.9977147528524714

In [66]:
# correr funcion para calcular probabilidades a futuro

rango_analisis = 15
dia_final = 59
rango_estimacion = 30
y_pred, y_pred_accum = calcular_probas(data_ventas, rango_analisis, dia_final, rango_estimacion)

In [67]:
y_pred

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
sku,,,,,,,,,,,,,,,,,,,,,
0,0.322,0.245,0.221,0.136,0.057,0.016,0.003,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.452,0.177,0.153,0.108,0.063,0.030,0.012,0.004,0.001,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.493,0.102,0.096,0.084,0.069,0.054,0.039,0.026,0.016,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.493,0.102,0.096,0.084,0.069,0.054,0.039,0.026,0.016,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.500,0.341,0.136,0.021,0.001,0.000,0.000,0.000,0.000,0.000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.010,0.042,0.116,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,0.486,0.102,0.097,0.085,0.071,0.055,0.040,0.027,0.017,0.010,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,0.472,0.141,0.127,0.101,0.071,0.044,0.024,0.012,0.005,0.002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [68]:
y_pred_accum

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
sku,,,,,,,,,,,,,,,,,,,,,
0,0.322,0.567,0.788,0.924,0.980,0.996,1.000,1.000,1.000,1.000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,0.452,0.629,0.782,0.890,0.953,0.983,0.995,0.999,1.000,1.000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,0.493,0.595,0.691,0.776,0.845,0.899,0.937,0.963,0.980,0.989,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,0.493,0.595,0.691,0.776,0.845,0.899,0.937,0.963,0.980,0.989,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,0.500,0.841,0.977,0.999,1.000,1.000,1.000,1.000,1.000,1.000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99996,0.000,0.000,0.000,0.000,0.000,0.000,0.002,0.012,0.055,0.170,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99997,0.486,0.589,0.685,0.770,0.841,0.896,0.935,0.962,0.979,0.989,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
99998,0.472,0.612,0.740,0.841,0.912,0.957,0.981,0.992,0.997,0.999,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [69]:
y_pred.sum(axis = 1).mean()

0.9971420185798141

In [70]:
y_pred.sum(axis = 1)

sku
0         1.000
1         1.000
2         0.999
3         0.999
4         0.999
          ...  
99996     0.999
99997     1.001
99998     1.000
99999     0.999
100000    0.999
Length: 100001, dtype: float64